# ZCRT

In [1]:
import cantera as ct
import numpy as np
import pandas as pd
from counterflow_file import *

In [2]:
#models = ['IEM','EMST','IEMHYB','EMSTHYB']
models = ['IEMHYB',]
params = {}
params['MIX'] = None
params['tres'] = -3.5
params['tmix'] = 0.2
params['eqv'] = 1.0

op_name = 'pasrfull.op'

In [3]:
Z_f = np.genfromtxt('template/flame_Z.dat',skip_header=1,usecols=(0,))
Z_lb = np.arange(0,Z_f.min(),0.004)
Z_ub = np.arange(Z_f.max(),0.2,0.005)
Z_ub = np.delete(Z_ub,0)
Z_r = np.arange(0.22,1,0.02)
Z_bins = np.concatenate((Z_lb,Z_f,Z_ub,Z_r,[1,]))

Z_mid = (Z_bins[1:]+Z_bins[:-1])/2
num_bins = len(Z_mid)

In [4]:
gas = ct.Solution('gri30.xml')
element_index = np.empty([len(gas.species_names),len(gas.element_names)])
for i, species in enumerate(gas.species_names):
    for j, element in enumerate(gas.element_names):
        element_index[i,j] = gas.n_atoms(species,element)

In [5]:
# Bilger's mixture fraction
coeff_Z = ( 2 * element_index[:,gas.element_index('C')]
            + element_index[:,gas.element_index('H')] / 2
            - element_index[:,gas.element_index('O')] )

OXY = {'O2':1, 'N2':3.76}
FUEL = {'CH4':1}

T_OXY = 300.
T_FUEL = 300.

p = 1. * ct.one_atm

# oxidizer stream
gas.TPX = T_OXY, p, OXY
# specific mole number
z_OXY = gas.Y/gas.molecular_weights
Z_OXY = np.dot(coeff_Z,z_OXY)

# fuel stream
gas.TPX = T_FUEL, p, FUEL
# specific mole number
z_FUEL = gas.Y/gas.molecular_weights
Z_FUEL = np.dot(coeff_Z,z_FUEL)

# coefficients for the progress variable
C_species = ['CO2','CO','H2O','H2']
coeff_C = np.zeros(gas.n_species)
for spe in C_species:
    spe_index = gas.species_index(spe)
    coeff_C[spe_index] = gas.molecular_weights[spe_index]

In [6]:
for model in models:
    params['MIX'] = model
    
    case = params2name(params)

    file_name = '/'.join([case,op_name])
    data = np.genfromtxt(file_name)
        
    Z = np.dot(coeff_Z,data[:,-gas.n_species:].T)
    Z = (Z-Z_OXY)/(Z_FUEL-Z_OXY)
        
    C = np.dot(coeff_C,data[:,-gas.n_species:].T)
    
    df = pd.DataFrame({
            'Z':Z,
            'C':C,
            'T':data[:,3],
            'R':data[:,1]})
    
    df.to_csv('/'.join([case,'ZCTR.csv']),index=False)